In [ ]:
## summarize("https://arxiv.org/pdf/2106.09685")- wont run as this needs pypdf library to scrape pdf

In [1]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
# If you get a wordnet error, also run:
nltk.download('wordnet')


from PyPDF2 import PdfReader
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist

def summarize_pdf(pdf_path, num_sentences=3):
    """
    Summarizes a PDF file.

    Args:
        pdf_path (str): The path to the PDF file.
        num_sentences (int, optional): The number of sentences in the summary. Defaults to 3.

    Returns:
        str: The summary of the PDF.
    """
    # Extract text from PDF
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()

    # Tokenize sentences
    sentences = sent_tokenize(text)

    # Clean and preprocess text
    stop_words = set(stopwords.words("english"))
    words = [word.lower() for sentence in sentences for word in sentence.split() if word.isalpha() and word.lower() not in stop_words]

    # Calculate word frequencies
    word_frequencies = FreqDist(words)

    # Score sentences based on word frequencies
    sentence_scores = {}
    for i, sentence in enumerate(sentences):
        for word in sentence.split():
            if word.lower() in word_frequencies:
                if i not in sentence_scores:
                    sentence_scores[i] = word_frequencies[word.lower()]
                else:
                    sentence_scores[i] += word_frequencies[word.lower()]

    # Get top n sentences
    top_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:num_sentences]

    # Create summary
    summary_sentences = [sentences[i] for i in top_sentences]
    summary = " ".join(summary_sentences)

    return summary

if __name__ == "__main__":
    pdf_file = "/Users/rojapalla/Documents/LORA.pdf"  # Replace with your PDF file path
    summary = summarize_pdf(pdf_file)
    print(summary)

[nltk_data] Downloading package punkt to /Users/rojapalla/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/rojapalla/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rojapalla/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rojapalla/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Some prior works
even explicitly impose the low-rank constraint when training the original neural network (Sainath
et al., 2013; Povey et al., 2018; Zhang et al., 2014; Jaderberg et al., 2014; Zhao et al., 2016; Kho-
dak et al., 2021; Denil et al., 2014); however, to the best of our knowledge, none of these works
considers low-rank update to a frozen model for adaptation to downstream tasks . Using language modeling as an example, there
are two prominent strategies when it comes to efﬁcient adaptations: adding adapter layers (Houlsby
et al., 2019; Rebufﬁ et al., 2017; Pfeiffer et al., 2021; R ¨uckl´e et al., 2020) or optimizing some forms
of the input layer activations (Li & Liang, 2021; Lester et al., 2021; Hambardzumyan et al., 2020;
Liu et al., 2021). Consider the following thought
experiment: if the downstream task were in a different language than the one used for pre-training, retraining
the entire model (similar to LoRA with r=dmodel ) could certainly outperform LoRA with a smal

In [2]:
import nltk
import string
from PyPDF2 import PdfReader
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer

# Download required resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def summarize_pdf(pdf_path, num_sentences=3):
    # Read the PDF
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + " "

    # Tokenize into sentences
    sentences = sent_tokenize(text)

    # Prepare stopwords and lemmatizer
    stop_words = set(stopwords.words("english"))
    lemmatizer = WordNetLemmatizer()

    # Clean and tokenize text
    words = []
    for sentence in sentences:
        for word in word_tokenize(sentence.lower()):
            if word.isalpha() and word not in stop_words:
                lemma = lemmatizer.lemmatize(word)
                words.append(lemma)

    # Compute word frequency
    word_freq = FreqDist(words)

    # Score sentences
    sentence_scores = {}
    for i, sentence in enumerate(sentences):
        sentence_words = word_tokenize(sentence.lower())
        sentence_score = 0
        for word in sentence_words:
            if word.isalpha():
                word = lemmatizer.lemmatize(word)
                if word not in stop_words:
                    sentence_score += word_freq[word]
        sentence_scores[i] = sentence_score

    # Pick top scoring sentences
    top_indices = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:num_sentences]
    top_indices.sort()  # Preserve original order

    summary = " ".join([sentences[i] for i in top_indices])
    return summary

if __name__ == "__main__":
    pdf_file = "/Users/rojapalla/Documents/LORA.pdf"  # Update as needed
    summary = summarize_pdf(pdf_file, num_sentences=3)
    print("\nSummary:\n")
    print(summary)


[nltk_data] Downloading package punkt to /Users/rojapalla/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rojapalla/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rojapalla/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



Summary:

Using language modeling as an example, there
are two prominent strategies when it comes to efﬁcient adaptations: adding adapter layers (Houlsby
et al., 2019; Rebufﬁ et al., 2017; Pfeiffer et al., 2021; R ¨uckl´e et al., 2020) or optimizing some forms
of the input layer activations (Li & Liang, 2021; Lester et al., 2021; Hambardzumyan et al., 2020;
Liu et al., 2021). RoB base(FT)* 125.0M 87.6 94.8 90.2 63.6 92.8 91.9 78.7 91.2 86.4
RoB base(BitFit)* 0.1M 84.7 93.7 92.7 62.0 91.8 84.0 81.5 90.8 85.2
RoB base(AdptD)* 0.3M 87.1.094.2.188.51.160.8.493.1.190.2.071.52.789.7.384.4
RoB base(AdptD)* 0.9M 87.3.194.7.388.4.162.6.993.0.290.6.075.92.290.3.185.4
RoB base(LoRA) 0.3M 87.5.395.1.289.7.763.41.293.3.390.8.186.6.791.5.287.2
RoB large(FT)* 355.0M 90.2 96.4 90.9 68.0 94.7 92.2 86.6 92.4 88.9
RoB large(LoRA) 0.8M 90.6.296.2.590.91.268.21.994.9.391.6.187.42.592.6.289.0
RoB large(AdptP)y 3.0M 90.2.396.1.390.2.768.31.094.8.291.9.183.82.992.1.788